# PSS

PSS(Parameter-Space Saliency)は，Deep Learningモデルの顕著性(Saliency)を可視化する手法の一つである．

誤分類に影響したパラメータを分析するアプローチで，影響の大きいパラメータを補正することでDeep Learningモデルの性能を改善できることが示された．

## 参考文献

* 論文
  * https://arxiv.org/abs/2108.01335
* GitHub
  * https://github.com/LevinRoman/parameter-space-saliency
* 解説資料：DL輪読会
  * https://www.slideshare.net/DeepLearningJP2016/dlwhere-do-models-go-wrong-parameterspace-saliency-maps-for-explainability

## PSS効果例

論文ではGrad-CAMとの比較が示されている．  
下図のようにPSSではGrad-CAMでは表現されない可視化要因を表現することができる(右2枚，ユキヒメドリ(junco)及び旅客列車(passenger car))．

![paper figure19](./figure/PSS/paper_figure19.png)